In [ ]:
from torch import nn
import torch.nn.functional as F
import numpy as np
import torchvision
import torchvision.models as models
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.transforms.functional import crop
import cv2
import matplotlib.pyplot as plt
import time
import torch

In [ ]:
def crop_r(image):
    return crop(image, 4, 124, 124, 124)
    
def crop_l(image):
    return crop(image, 4, 4, 124, 124)  
    
transform_right = transforms.Compose(
    [
        transforms.Resize((256,256)),
        #transforms.RandomRotation(10),
        transforms.Lambda(crop_r),
        transforms.ToTensor(),
        transforms.Normalize((0.36, 0.36, 0.36), (0.28, 0.28, 0.28))
    ])
transform_left = transforms.Compose(
    [
        transforms.Resize((256,256)),
        #transforms.RandomRotation(10),
        transforms.Lambda(crop_l),
        transforms.ToTensor(),
        transforms.Normalize((0.36, 0.36, 0.36), (0.28, 0.28, 0.28))
    ])

In [ ]:
train_path = '/content/drive/MyDrive/Datasets/TUM'
batch_size = 1

train_r = datasets.ImageFolder(train_path, transform_right)
train_l = datasets.ImageFolder(train_path, transform_left)

train_loader_r = torch.utils.data.DataLoader(train_r, batch_size)
train_loader_l = torch.utils.data.DataLoader(train_l, batch_size)


In [ ]:
weights = models.ConvNeXt_Base_Weights
model = models.convnext_base(weights=weights)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Base_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Base_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/convnext_base-6075fbad.pth" to /root/.cache/torch/hub/checkpoints/convnext_base-6075fbad.pth


  0%|          | 0.00/338M [00:00<?, ?B/s]

In [ ]:
model

ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((128,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (1): Permute()
          (2): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=128, out_features=512, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=512, out_features=128, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
          (1): Permute()
          (2): LayerNorm((128,), eps=1e-06, elementwise_affine=True)
          (3): Linear(

In [ ]:
def rightpt(train_loader_r, min_, max_, aux_):
  for i, data in enumerate(train_loader_r):
      if i > min_:
        inputs, labels = data
        outputs = model.classifier[0](model.avgpool(model.features(inputs)))
        output_.append(outputs)
        if i > max_:
          return output_


In [ ]:
def leftpt(train_loader_l, min_, max_, aux_,output_):
  for i,data in enumerate(train_loader_l):
    if i > min_:
      inputs, labels = data
      outputs = model.classifier[0](model.avgpool(model.features(inputs)))
      output_[i-aux_] = torch.cat((output_[i-aux_], outputs), 1)
      if i > max_:
          return output_
           

In [ ]:
for n_it in range(1,169):
  output_ = []
  output_new = []
  try:
    if n_it == 1:
      min_ = 0
    else:
      min_ = (n_it-1)*50
    max_ = n_it*50
    aux_ = min_ +1
    output_ = rightpt(train_loader_r, min_, max_, aux_)
    output_new = leftpt(train_loader_l, min_, max_, aux_, output_)
    out_path = '/content/drive/MyDrive/Datasets/tensors_tum_convnext/tensors'+str(n_it)+'.pt'
    torch.save(output_new,out_path)
  except:
    print('error')


In [ ]:
def match(cam_img, db_img, dist_limit):
    distance = torch.cdist(cam_img, db_img, p=2)
    max_dist = torch.max(distance)
    if max_dist <= dist_limit:
        return True
    
    return False

In [ ]:
def belief_generation(db, descriptor):
    belief = []
    lp = 0
    for d in db:
        bg = match(descriptor, d, 6)
        if bg:
            belief.append(1)
        else:
            belief = []
        if len(belief) >= 200:
            print('loop')
            print(d)
            lp += 1
    return lp

In [ ]:
database = [database1,database2,database3,database4,database5,database6,database7,database8,database9,database10,
            database11,database12,database13,database14,database15,database16,database17,database18,database19,database20,
            database21,database22,database23,database24,database25,database26,database27,database28,database29,database30,
            database31,database32,database33,database34,database35,database36,database37,database38,database39,database40,
            database41,database42,database43,database44,database45,database46,database47,database48,database49,database50,
            database51,database52,database53,database54,database55,database56,database57,database58,database59,database60,
            database61,database62,database63,database64,database65,database66,database67,database68,database69,database70,
            database71,database72,database73,database74,database75,database76,database77,database78,database79,database80,
            database81,database82,database83,database84,database85,database86,database87,database88,database89,database90,
            database91,database92,database93,database94,database95,database96,database97,database98,database99,database100,
            database101,database102,database103,database104,database105,database106,database107,database108,database109,database110,
            database111,database112,database113,database114,database115,database116,database117,database118,database119,database120,
            database121,database122,database123,database124,database125,database126,database127,database128,database129,database130,
            database131,database132,database133,database134,database135,database136,database137,database138,database139,database140,
            database141,database142,database143,database144,database145,database146,database147,database148,database149,database150,
            database151,database152,database153,database154,database155,database156]
d = []

In [ ]:
co = 0
for bg in database:
  for i,data in enumerate(bg):
    if i < 3:
      d.append(data)
    else:
      loop_ = belief_generation(d, data)
      if loop_ > 0:
        with open('/content/drive/MyDrive/Datasets/TUM_/loop_tum_convnext_v1.txt', 'a') as txt_file:
          txt_file.write('\n Found loop')
          image_number = (co*50)+i
          txt_file.write('\n image: '+str(image_number))
      else:
        d.append(data)
  co += 1


A saída de streaming foi truncada nas últimas 5000 linhas.

         [[ 1.0120]]]], requires_grad=True)
loop
tensor([[[[ 0.0969]],

         [[-0.3488]],

         [[-0.5110]],

         ...,

         [[-0.2679]],

         [[ 0.4123]],

         [[ 0.6797]]]], requires_grad=True)
loop
tensor([[[[ 0.0488]],

         [[-0.3796]],

         [[-0.9803]],

         ...,

         [[ 0.0726]],

         [[ 0.2578]],

         [[ 1.3853]]]], requires_grad=True)
loop
tensor([[[[ 0.3224]],

         [[-0.3681]],

         [[-0.7378]],

         ...,

         [[-0.0467]],

         [[ 0.2026]],

         [[ 1.5366]]]], requires_grad=True)
loop
tensor([[[[-0.1371]],

         [[-0.1341]],

         [[-0.9948]],

         ...,

         [[-0.0734]],

         [[ 0.3368]],

         [[ 0.9631]]]], requires_grad=True)
loop
tensor([[[[ 0.0846]],

         [[-0.5394]],

         [[-0.8079]],

         ...,

         [[ 0.0310]],

         [[ 0.4606]],

         [[ 0.6553]]]], requires_grad=True)
l

In [ ]:
c = 0
for d in database74:
    distance = torch.cdist(d, database71[10], p=2)
    max_dist = torch.max(distance)
    print('image: {}, distance: {}'.format(c, max_dist))
    c += 1

image: 0, distance: 6.187118053436279
image: 1, distance: 5.69697904586792
image: 2, distance: 3.8911361694335938
image: 3, distance: 6.928799629211426
image: 4, distance: 6.001237869262695
image: 5, distance: 4.99795389175415
image: 6, distance: 5.992709159851074
image: 7, distance: 5.3638105392456055
image: 8, distance: 6.324755668640137
image: 9, distance: 4.3462677001953125
image: 10, distance: 7.874664783477783
image: 11, distance: 7.88560152053833
image: 12, distance: 5.122623920440674
image: 13, distance: 3.2841405868530273
image: 14, distance: 3.9971706867218018
image: 15, distance: 5.203649044036865
image: 16, distance: 6.249173641204834
image: 17, distance: 6.710351943969727
image: 18, distance: 7.498437404632568
image: 19, distance: 4.473146915435791
image: 20, distance: 5.98167610168457
image: 21, distance: 4.097232341766357
image: 22, distance: 4.658495903015137
image: 23, distance: 4.859636306762695
image: 24, distance: 3.719546318054199
image: 25, distance: 5.23333406448

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Datasets/metricas/tum_convnext.csv')
